In [7]:
import pandas as pd
import numpy as np

In [4]:
%pwd

'c:\\Users\\shipr\\OneDrive\\Documents\\project\\Vaccine_Data_Analysis'

In [5]:
import os

In [2]:
os.chdir('..')

In [8]:
sch = pd.read_excel("raw/vaccine-schedule-data.xlsx")

In [9]:
sch.head()

,ISO_3_CODE,COUNTRYNAME,WHO_REGION,YEAR,VACCINECODE,VACCINE_DESCRIPTION,SCHEDULEROUNDS,TARGETPOP,TARGETPOP_DESCRIPTION,GEOAREA,AGEADMINISTERED,SOURCECOMMENT
0,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,1.0,NaN,General/routine,NATIONAL,M2,NaN
1,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,2.0,NaN,General/routine,NATIONAL,M4,NaN
2,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,3.0,NaN,General/routine,NATIONAL,M6,NaN
3,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,4.0,B_2YL_W,General/routine,NATIONAL,M15,NaN
4,ABW,Aruba,AMRO,2023.0,DTAPIPV,DTaP-IPV (acellular) vaccine,5.0,B_CHILD_W,General/routine,NATIONAL,Y4,NaN


In [10]:
sch.isnull().sum()

ISO_3_CODE                  0
COUNTRYNAME                 1
WHO_REGION                  1
YEAR                        1
VACCINECODE                 1
VACCINE_DESCRIPTION         1
SCHEDULEROUNDS              1
TARGETPOP                4258
TARGETPOP_DESCRIPTION       1
GEOAREA                    31
AGEADMINISTERED          1046
SOURCECOMMENT            2914
dtype: int64

In [11]:
sch['SOURCECOMMENT'].value_counts()

SOURCECOMMENT
0-11 mois                                                                                                               185
Pregnant women                                                                                                          157
Health and care workers                                                                                                 124
Children with chronic conditions                                                                                         97
0-11 months                                                                                                              91
                                                                                                                       ... 
outrbreak response                                                                                                        1
Girls aged 9-14 years old (in school and out of school) reached through an HPV Multi Age Cohort Vaccination Campaign  

In [12]:
sch['SOURCECOMMENT'].isnull().sum()

np.int64(2914)

In [11]:
import pandas as pd
import numpy as np

# Load data


print("Pure Pandas Imputation Solution")
print("="*60)

# Function to impute using pandas only
def pandas_impute(sch):
    df_imputed = sch.copy()
    
    # 1. GEOAREA - always NATIONAL
    df_imputed['GEOAREA'] = df_imputed['GEOAREA'].fillna('NATIONAL')
    
    # 2. AGEADMINISTERED - vaccine-specific rules
    def get_age(vaccine, round_num):
        rules = {
            'BCG': {1: 'B'},
            'HEPB_PEDIATRIC': {1: 'B', 2: 'M1', 3: 'M6'},
            'DTWPHIBHEPB': {1: 'M2', 2: 'M4', 3: 'M6'},
            'OPV': {1: 'B', 2: 'W6', 3: 'W10', 4: 'W14', 5: 'M9'},
            'IPV': {1: 'M4', 2: 'M9'},
            'MMR': {1: 'M12', 2: 'Y4'},
            'PCV13': {1: 'M2', 2: 'M4', 3: 'M12'},
            'HPV4': {1: 'Y11', 2: 'Y12'},
            'TD_S': {1: '1st contact', 2: '+M1', 3: '+M6', 4: '+Y1', 5: '+Y1'},
        }
        return rules.get(vaccine, {}).get(round_num, f'Round {round_num}')
    
    mask = df_imputed['AGEADMINISTERED'].isna()
    df_imputed.loc[mask, 'AGEADMINISTERED'] = df_imputed[mask].apply(
        lambda x: get_age(x['VACCINECODE'], x['SCHEDULEROUNDS']), axis=1
    )
    
    # 3. TARGETPOP - description-based + lookup
    # First pass: fill from similar records
    for idx, row in df_imputed[df_imputed['TARGETPOP'].isna()].iterrows():
        similar = df_imputed[
            (df_imputed['COUNTRYNAME'] == row['COUNTRYNAME']) &
            (df_imputed['VACCINECODE'] == row['VACCINECODE']) &
            (df_imputed['TARGETPOP_DESCRIPTION'] == row['TARGETPOP_DESCRIPTION']) &
            (df_imputed['TARGETPOP'].notna())
        ]
        if not similar.empty:
            df_imputed.at[idx, 'TARGETPOP'] = similar['TARGETPOP'].mode().iloc[0]
    
    # Second pass: description mapping
    desc_map = {
        'General/routine': 'GENERAL',
        'Travellers': 'TRAVELLERS',
        'Health workers': 'HW',
        'Pregnant women': 'PW',
        'Risk group(s)': 'RISKGROUPS',
        'HPV for females only': 'FEMALE',
        'Adults': 'ADULTS',
        'Catch-up adults': 'CATCHUP_A',
        'Catch-up children': 'CATCHUP_C',
        'HPV administered to females and males': 'BOTH'
    }
    
    df_imputed['TARGETPOP'] = df_imputed.apply(
        lambda x: desc_map.get(x['TARGETPOP_DESCRIPTION'], 'OTHER') 
        if pd.isna(x['TARGETPOP']) else x['TARGETPOP'],
        axis=1
    )
    
    # 4. SOURCECOMMENT - descriptive fill
    df_imputed['SOURCECOMMENT'] = df_imputed.apply(
        lambda x: f"{x['TARGETPOP_DESCRIPTION']} in {x['COUNTRYNAME']} ({x['YEAR']})" 
        if pd.isna(x['SOURCECOMMENT']) else x['SOURCECOMMENT'],
        axis=1
    )
    
    return df_imputed

# Apply imputation
df_clean = pandas_impute(sch)

print("\nMissing values after pandas imputation:")
print(df_clean[['TARGETPOP', 'SOURCECOMMENT', 'AGEADMINISTERED', 'GEOAREA']].isnull().sum())

# Save
df_clean.to_csv('vaccination_data_pandas_imputed.csv', index=False)
print("\n✅ Saved to: vaccination_data_pandas_imputed.csv")

Pure Pandas Imputation Solution

Missing values after pandas imputation:
TARGETPOP          0
SOURCECOMMENT      0
AGEADMINISTERED    0
GEOAREA            0
dtype: int64

✅ Saved to: vaccination_data_pandas_imputed.csv


In [12]:
df_clean.head()

,ISO_3_CODE,COUNTRYNAME,WHO_REGION,YEAR,VACCINECODE,VACCINE_DESCRIPTION,SCHEDULEROUNDS,TARGETPOP,TARGETPOP_DESCRIPTION,GEOAREA,AGEADMINISTERED,SOURCECOMMENT
0,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,1.0,B_2YL_W,General/routine,NATIONAL,M2,General/routine in Aruba (2023.0)
1,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,2.0,B_2YL_W,General/routine,NATIONAL,M4,General/routine in Aruba (2023.0)
2,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,3.0,B_2YL_W,General/routine,NATIONAL,M6,General/routine in Aruba (2023.0)
3,ABW,Aruba,AMRO,2023.0,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,4.0,B_2YL_W,General/routine,NATIONAL,M15,General/routine in Aruba (2023.0)
4,ABW,Aruba,AMRO,2023.0,DTAPIPV,DTaP-IPV (acellular) vaccine,5.0,B_CHILD_W,General/routine,NATIONAL,Y4,General/routine in Aruba (2023.0)


In [13]:
df_clean.isnull().sum()

ISO_3_CODE               0
COUNTRYNAME              1
WHO_REGION               1
YEAR                     1
VACCINECODE              1
VACCINE_DESCRIPTION      1
SCHEDULEROUNDS           1
TARGETPOP                0
TARGETPOP_DESCRIPTION    1
GEOAREA                  0
AGEADMINISTERED          0
SOURCECOMMENT            0
dtype: int64

In [14]:
df_clean = df_clean[~df_clean['ISO_3_CODE'].astype(str).str.contains('Created:', na=False)]

In [15]:
df_clean.isnull().sum() 

ISO_3_CODE               0
COUNTRYNAME              0
WHO_REGION               0
YEAR                     0
VACCINECODE              0
VACCINE_DESCRIPTION      0
SCHEDULEROUNDS           0
TARGETPOP                0
TARGETPOP_DESCRIPTION    0
GEOAREA                  0
AGEADMINISTERED          0
SOURCECOMMENT            0
dtype: int64

In [16]:
df_clean.dtypes

ISO_3_CODE                object
COUNTRYNAME               object
WHO_REGION                object
YEAR                     float64
VACCINECODE               object
VACCINE_DESCRIPTION       object
SCHEDULEROUNDS           float64
TARGETPOP                 object
TARGETPOP_DESCRIPTION     object
GEOAREA                   object
AGEADMINISTERED           object
SOURCECOMMENT             object
dtype: object

In [18]:
df_clean['YEAR'] = pd.to_numeric(df_clean['YEAR'], errors='coerce').astype('Int64')
df_clean['SCHEDULEROUNDS'] = pd.to_numeric(df_clean['SCHEDULEROUNDS'], errors='coerce').astype('Int64')

In [19]:
df_clean.dtypes

ISO_3_CODE               object
COUNTRYNAME              object
WHO_REGION               object
YEAR                      Int64
VACCINECODE              object
VACCINE_DESCRIPTION      object
SCHEDULEROUNDS            Int64
TARGETPOP                object
TARGETPOP_DESCRIPTION    object
GEOAREA                  object
AGEADMINISTERED          object
SOURCECOMMENT            object
dtype: object

In [20]:
df_clean.head()

,ISO_3_CODE,COUNTRYNAME,WHO_REGION,YEAR,VACCINECODE,VACCINE_DESCRIPTION,SCHEDULEROUNDS,TARGETPOP,TARGETPOP_DESCRIPTION,GEOAREA,AGEADMINISTERED,SOURCECOMMENT
0,ABW,Aruba,AMRO,2023,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,1,B_2YL_W,General/routine,NATIONAL,M2,General/routine in Aruba (2023.0)
1,ABW,Aruba,AMRO,2023,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,2,B_2YL_W,General/routine,NATIONAL,M4,General/routine in Aruba (2023.0)
2,ABW,Aruba,AMRO,2023,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,3,B_2YL_W,General/routine,NATIONAL,M6,General/routine in Aruba (2023.0)
3,ABW,Aruba,AMRO,2023,DTAPHIBIPV,DTaP-Hib-IPV (acellular) vaccine,4,B_2YL_W,General/routine,NATIONAL,M15,General/routine in Aruba (2023.0)
4,ABW,Aruba,AMRO,2023,DTAPIPV,DTaP-IPV (acellular) vaccine,5,B_CHILD_W,General/routine,NATIONAL,Y4,General/routine in Aruba (2023.0)


In [21]:
df_clean =df_clean.to_excel("processed/cleaned_vaccine_schedule_data.xlsx", index=False)